Universidad del Valle de Guatemala <br>
Facultad de Ingeniería <br>
Departamento de Ciencias de la computación <br>
Inteligencia Artificial 

# Laboratorio 10

## Integrantes
Christopher García <br>
Ma. Isabel Solano <br>

## Task 1 - Clasificador de Imágenes (Perros vs Gatos)
Deberá construir un modelo basado en redes neuronales que le permita clasificar perros y gatos basado en un input de imágen. Para esto, deberá usar técnicas basadas en Deep Learning relacionadas con el manejo de imágenes como las redes  convolucionales. Esto lo deberá hacer usando su librería de preferencia como PyTorch, Tensoflow
entre otras. Recuerden que: 
- Deben hacer una breve exploración con los datos. Esto implica, pero no está limitado a:
    - Limpieza de imágenes
    - Escalamiento de las imágenes
    - Estandarización de color
    - Manipulación de imágenes
        - Agregar contorno
        - Cambiar dirección
    - Revisar si el dataset está balanceado, caso no estarlo, aplicar alguna técnica para balancearlo lo más y mejor posible
- Recuerden hacer el split para training, testing y si consideran necesario para validation
    - 80% training
    - 20% testing
        - 10% validation si lo necesitan
- Recuerde definir de forma clara y razonada (es decir, diga el por qué de su elección) de una métrica de desempeño principal

### Task 1.1 - Lectura del Dataset
En esta ocasión no recibirá un documento único como entrada de datos, por el contrario deberá recibir un conjunto de imágenes que representan perros y gatos. Estos datos los puede descargar de este dataset de Kaggle. Deberá leer estas imágenes para crear un dataset en memoria que haga referencia a la imagen con sus características y a la categoría a la que pertenece. Recuerde que debe dividir su dataset según el uso y como se mencionó anteriormente. Además, considere que la cantidad de imágenes de perros y gatos puede estar desigual por lo que deberá aplicar las técnicas aprendidas durante el curso para lidiar con este problema Son libres de usar todas las imágenes del dataset dado o bien una cantidad definida por ustedes mismos siempre y cuando se argumente la razón de forma debida.

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms


In [30]:
import os

path = "PetImages"  # Replace this with the actual path to the directory
num_elements = len(os.listdir(path))

print("Number de categorías:", num_elements)
print("Cantidad de perros: ", len(os.listdir("PetImages/Dog")))
print("Cantidad de gatos: ", len(os.listdir("PetImages/Cat")))


Number de categorías: 2
Cantidad de perros:  12501
Cantidad de gatos:  12501


### Task 1.2 - Construcción del Modelo
Deberá crear un modelo de Deep Learning orientado a imágenes, como una red que usa capas convolucionales, para poder resolver este laboratorio. Recuerde que deberá definir la arquitectura de su red, aplicando las diferentes técnicas vistas en clase (dropouts, funciones de activación, padding, stride, etc). Podrá usar la librería que más le parezca para completar el laboratorio.


In [31]:
# definiendo transformers
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [32]:
data_dir = "PetImages"  # Replace this with the actual path to the dataset
image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms)
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=4, shuffle=True)


In [33]:
# Definiendo el modelo, es normal que este dergargue cosas

model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)


In [34]:
# Definición de función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [35]:
# Entrenamiento
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloaders, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 0.565
[1,   400] loss: 0.414
[1,   600] loss: 0.311
[1,   800] loss: 0.342
[1,  1000] loss: 0.353
[1,  1200] loss: 0.345
[1,  1400] loss: 0.305
[1,  1600] loss: 0.330
[1,  1800] loss: 0.260
[1,  2000] loss: 0.296
[1,  2200] loss: 0.313
[1,  2400] loss: 0.213
[1,  2600] loss: 0.189
[1,  2800] loss: 0.153
[1,  3000] loss: 0.148
[1,  3200] loss: 0.114


UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='PetImages\\Cat\\666.jpg'>

### Task 1.3 - Desempeño del modelo
Al finalizar el entrenamiento de su modelo, permita que se ingresen nuevas imágenes y permita que el modelo la clasifique. Además recuerde medir el desempeño de su modelo tanto en entrenamiento como en testing. Para ello, deberá mostrar las métricas de desempeño de su modelo para las fases dichas, además de la evolución de las métricas durante las diferentes épocas, cuidando siempre no hacer overfitting sobre el dataset.


In [ ]:
# prueba de desempeño 
correct = 0
total = 0
with torch.no_grad():
    for data in dataloaders:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

